In [1]:
# use the cell below to ensure correct relative imports
# you must be in the qcore directory

In [2]:
cd ../qcore

C:\Users\athar\Desktop\qcore\qcore


### Proto-user guide for interacting with the Stage object in qcore

Motivation: a way to control all Instruments that you're interacting with as part of an experiment.

Definition: an experimental stage is a group of instruments needed to run a given experiment. These instruments may be physical hardware or may be meta-instruments (e.g. qubit, rr)

1. With Stage, you can initialize all your instruments from a single yaml file. Initialize means the instrument is connected, set with initial parameters, and ready for use!
2. You can add (`enter`) or remove (`exit`) instruments from the stage. You can also `enter` instruments that are saved in yaml files!
3. You can get current state of all the instruments in the stage using `stage.parameters` !!!

## Goal - we will build up the experimental stage that we're currently using to characterize a 1 qubit, 1 rr device with opx, sa, labbricks

### After we do so, we'll save it as a yaml file so that the next time we can simply load it from that yaml file and we'll be good to go!!!

Note - as of the writing of this tutorial, the mixer tuning step has not yet been developed and integrated with the rest of the codebase, so we'll skip that for now, and maybe do another tutorial incorporating it later :)

In [4]:
from pathlib import Path

#### Step 1 - Create the QuantumDevice

In [5]:
# we have already done this step in the cqed_components tutorial
# we saved the device in 'example_device.yaml' in this folder
# let's create a variable that remembers this path

device_yaml_path = Path.cwd().parent / 'tutorials/example_device.yaml'

#### Step 2 - Encapsulate the readout resonator as a QuantumElement

#### Step 3 - Wrap the two QuantumElements in a QuantumDevice

#### Step 4 - 